In [ ]:
# !pip install python-dotenv
# !pip install python-telegram-bot

In [1]:
%pip install telegram
%pip install python-telegram-bot
%pip install langchain
%pip install langchain-community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [2]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tqdm
%pip install transformers
%pip install pymupdf


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install PyPDF2
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



In [5]:
import os
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import faiss
from tqdm import tqdm_notebook
from transformers import pipeline

import fitz  # PyMuPDF
from PyPDF2 import PdfReader

from dotenv import load_dotenv
load_dotenv()  

# https://huggingface.co/settings/tokens
API_TOKEN = os.getenv("HF_API_TOKEN")
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1"

# Чтение данных

## Лекции в pdf

In [6]:
def read_pdfs(pdf_dir):
    texts = []
    # Проходим по всем файлам в указанной директории и её поддиректориях
    for root, dirs, files in os.walk(pdf_dir):
        for file in tqdm_notebook(files):
            # Проверяем, что файл имеет расширение .pdf
            if file.endswith(".pdf"):
                # Извлекаем номер лекции из имени файла
                lecture_num = file.split('.')[0] if len(
                    file.split('.')) > 2 else ''
                pdf_path = os.path.join(root, file)
                try:
                    reader = PdfReader(pdf_path)
                except Exception as e:
                    print(f"Ошибка чтения файла {file}: {e}")
                    continue

                # Добавляем заголовок для текста, если номер лекции был извлечён
                text = f"Лекция №{lecture_num} конспект. " if lecture_num else ""

                # Извлекаем текст со всех страниц PDF
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                texts.append({"text": text, "source": file})
    return texts

In [7]:
pdf_texts = read_pdfs("C:/Users/saido/Desktop/Cute_Tutor/")

C:\Users\saido\AppData\Local\Temp\ipykernel_18864\3389196000.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm_notebook(files):


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
pdf_texts[:1]

[{'text': 'КлассичесĸаямеханиĸаĸонтрольныевопросыЛатыповаАделина216pр.2022-20231СффлизмруйтезаĸоныНьютонаВсегоЗзаĸона:дВсяĸоетелосохранлетсостояниепоĸояилиравномернойПрямолинейногодвижениядотСХПОР,поĸадругиетеланезаставатегоизменитьэтосостояние2ЗПроизведениемассыматериальнойточĸиНаУсĸорениеравнодействующейнонеесилетё-ĸЗ)действидвухтелдрyгнадругоиравныипротивоположнонаправленыFi--FлСила,действующаянаХсиладействующаянанеетелосоСтороны2-годетелоСОСтороны-гоЗзаĸонНьютона.ДвателадействуютдругнадрyгассиламиИРавнымипомодулю21ПротивоположнымипожаправлениюЗ1Однойприфодыв4)ПриложеннымиĸразнымжеламБУдействуютвдольоднойгрямойfI.Травитационноговзаимодействнястелаобладающциямассой)I.ЭлеĸтромагнитноговзаимодействияТСильноговзаимодействия(внутригдразедернаяфизинаĸСлабоговзаимодействияФсилытожЕстиFM=MgF=G,MR2OF=K9182агзаĸонКулона,Fтp.Тynp--ĸоx,T.N.FcoupТПротоныинейтроныдержатсяведреблагодаряСильнолувзаимодействиюПОНитринаБетла-распад2.ЧтотаĸоеСилаимасса?КаĸИХизмерить?Массо-мероатĸлиĸателаНадействиешилы

## Семинары в Jupyter Notebooks

In [9]:
from nbformat import read


def load_notebook_texts(nb_dir):
    texts = []
    for root, dirs, files in os.walk(nb_dir):
        for file in tqdm_notebook(files):
            if file.endswith(".ipynb"):
                notebook_path = os.path.join(root, file)
                try:
                    # Открываем и читаем содержимое ноутбука
                    with open(notebook_path, 'r', encoding='utf-8') as f:
                        nb = read(f, as_version=4)  # Читаем ноутбук в версии 4
                        notebook_content = {"source": file, "text": ""}
                        # Проходим по ячейкам ноутбука
                        for cell in nb.cells:
                            if cell.cell_type == 'markdown':
                                # Добавляем содержимое ячейки markdown
                                notebook_content["text"] += f"**Markdown:** \n{cell.source}\n\n"
                            elif cell.cell_type == 'code':
                                # Добавляем содержимое ячейки кода
                                notebook_content["text"] += f"**Code:** \n{cell.source}\n\n"
                        texts.append(notebook_content)
                except Exception as e:
                    print(f"Ошибка чтения файла {notebook_path}: {e}")
                    continue  # Продолжаем обработку других файлов

    return texts

In [10]:
notebook_texts = load_notebook_texts("seminar/")

In [11]:
notebook_texts[:1]

[]

## Лекции и семинары из видео в формате txt

In [ ]:
def load_lecture_texts(txt_dir):
    texts = []
    for root, dirs, files in os.walk(txt_dir):
        for file in tqdm_notebook(files):
            if file.endswith(".txt"):
                txt_content = {"source": os.path.join(root, file), "text": ""}
                try:
                    # Открытие файла в режиме чтения
                    with open(os.path.join(root, file), 'r') as f:
                        txt_content['text'] = f"{file.split('.txt')[0]} " + f.read()
                except Exception as e:
                    print(f"Ошибка чтения файла {file}: {e}")
                    continue
                texts.append(txt_content)
    return texts

In [ ]:
txt_video_lecture = load_lecture_texts("lecture_video/")

In [ ]:
txt_video_lecture[:1]

# Обработка данных

**LangChain** — это мощный фреймворк для создания приложений, использующих языковые модели (LLMs) для решения сложных задач. Он упрощает разработку и интеграцию приложений, которые работают с текстом, отвечают на вопросы, выполняют поиск, обрабатывают документы и реализуют диалоговые интерфейсы.

---
`RecursiveCharacterTextSplitter` — это инструмент из библиотеки LangChain, который используется для разбиения длинного текста на меньшие части (или фрагменты) определённого размера. Его ключевая особенность — рекурсивный подход к разбиению текста с использованием приоритетного списка разделителей.

Если текст слишком длинный для обработки языковой моделью, его нужно разбить на фрагменты (chunks). Инструмент разбивает текст с учётом логических границ (например, абзацев, предложений или слов), чтобы сохранить смысл текста

Используется список разделителей, которые применяются в порядке убывания приоритета Например:
- Сначала текст делится по двойным переводам строки (`\n\n`), чтобы выделить абзацы
- Если фрагмент всё ещё слишком длинный, используется обычный перевод строки (`\n`)
- Затем можно попробовать разбить текст по пробелам (` `)
-  Если и это не помогло, текст делится по символам

**Параметры:**
- `chunk_size` - максимальная длина одного фрагмента (например, 1000 символов).
- `chunk_overlap` - число символов, которое будет перекрывать соседние фрагменты. Это важно, чтобы не терять контекст, когда текст разбивается на части
- `separators` - список разделителей, которые используются для разбиения текста (например, `["\n\n", "\n", " ", ""]`)
---

`Document` — это класс из библиотеки **LangChain**, который используется для представления текстовой информации в формате, удобном для дальнейшей обработки. 

**Параметры:**
1. `page_content` - основной текст, который содержит содержание документа
2. `metadata` - словарь с дополнительной информацией (например, источник текста, номер страницы и т. д.).

Этот класс полезен для структурирования текста перед передачей в инструменты обработки, такие как модели машинного обучения или поисковые системы.

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def prepare_documents(data):
    # Настраиваем разбиение текста на части
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Максимальный размер одного фрагмента
        chunk_overlap=200,  # Перекрытие между фрагментами
        separators=["\n\n", "\n", " ",
                    ""]  # Приоритетные разделители для разбиения
    )
    documents = []
    # Разбиваем каждый текст на фрагменты
    for item in tqdm_notebook(data):
        splits = text_splitter.split_text(item["text"])  # Разбиваем текст
        for i, split in enumerate(splits):  # Создаём объект Document
            documents.append(
                Document(
                    page_content=split,  # Содержимое текста
                    metadata={"source": f"{item['source']}_part_{i}"
                              }  # Метаданные: источник и номер фрагмента
                ))
    return documents

In [13]:
#all_data = pdf_texts + notebook_texts + txt_video_lecture
all_data = pdf_texts
docs = prepare_documents(all_data)

C:\Users\saido\AppData\Local\Temp\ipykernel_18864\3218121964.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for item in tqdm_notebook(data):


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
docs[10]

Document(metadata={'source': 'controling_questions.pdf_part_10'}, page_content='салвобщемФоĸусаĸоторыжнаходитсяlСоянцеа)заравныепрометутĸивременирадиус-веĸ-ТОРпланетыачерчивастравныеплощадиrЗКвадратыпериодовоброщениепланетотносятаĸаĸĸубыболжнхполудийИЛэаиптичесĸийарбитГО.Чтотаĸосмоментиенерциитвердоготела?ПриведитепримерыМоментСенерилмитвердоготела-величинаапределяемаяформулойI-ExMiritрасстоянueданный')

# Создание векторов лекции и семинары

In [15]:
%pip install --upgrade langchain 

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install --force-reinstall transformers


  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.4.5-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached charset_n

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
gradio 5.6.0 requires markupsafe~=2.0, but you have markupsafe 3.0.2 which is incompatible.
nncf 2.14.0 requires networkx<=3.3,>=2.6, but you have networkx 3.4.2 which is incompatible.
nncf 2.14.0 requires numpy<2.2.0,>=1.19.1, but you have numpy 2.2.1 which is incompatible.
openvino 2024.5.0 requires numpy<2.2.0,>=1.16.6, but you have numpy 2.2.1 which is incompatible.


In [17]:
%pip install --force-reinstall sentence-transformers

  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached netwo

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
gradio 5.6.0 requires markupsafe~=2.0, but you have markupsafe 3.0.2 which is incompatible.
nncf 2.14.0 requires networkx<=3.3,>=2.6, but you have networkx 3.4.2 which is incompatible.
nncf 2.14.0 requires numpy<2.2.0,>=1.19.1, but you have numpy 2.2.1 which is incompatible.
openvino 2024.5.0 requires numpy<2.2.0,>=1.16.6, but you have numpy 2.2.1 which is incompatible.


In [18]:
import transformers
import sentence_transformers

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\saido\AppData\Local\Temp\ipykernel_18864\2766246740.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [19]:
from langchain.vectorstores import FAISS

def create_vector_store(documents, embeddings, vector_store_path="faiss_index"):
    """
    Создает векторное хранилище (FAISS) для документов, используя модель эмбеддингов HuggingFace
    """
    vector_store = None
    with tqdm_notebook(total=len(documents), desc="Ingesting documents") as pbar:
        for d in documents:
            if vector_store:
                # Если хранилище уже создано, добавляем документ
                vector_store.add_documents([d])
            else:
                # Если хранилище ещё не создано, создаем его и добавляем первый документ
                vector_store = FAISS.from_documents([d], embeddings)
            pbar.update(1)  # Обновляем прогресс

    # Сохраняем созданное векторное хранилище в локальный файл
    vector_store.save_local(vector_store_path)
    return vector_store

In [20]:
vector_store = create_vector_store(docs[:2], embeddings)

C:\Users\saido\AppData\Local\Temp\ipykernel_18864\2378222535.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm_notebook(total=len(documents), desc="Ingesting documents") as pbar:


Ingesting documents:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
def load_vector_store(embeddings, vector_store_path="faiss_index"):
    vector_store = FAISS.load_local(
        folder_path=vector_store_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True  # Убедитесь, что файл безопасен!
    )
    return vector_store

In [22]:
vector_store = load_vector_store(embeddings)

In [23]:
vector_store

# Запросы к Hugging Face API

## Поиск релевантных документов

In [43]:
def create_prompt(student_query, relevant_docs):
    # Создаем промт с учетом найденных документов
    docs_summary = " ".join([doc.metadata['source'] + doc.page_content for doc in relevant_docs])  # Объединение текста из найденных документов
    
    prompt = f"""
    Ты — образовательный ассистент (репетитор), помогающий ученикам углубить понимание различных тем.
    
    Когда ученик задаёт вопрос, твоя цель — решить его задачу, подробно объяснить решение и дать максимально полный ответ.
    
    Вот набор соответствующих документов для использования: {docs_summary}
    Вопрос ученика: {student_query}
    
    Предоставь конкретный ответ, а также объясни ключевые фундаментальные концепции, лежащие в его основе, и почему ответ именно такой.
    
    Если ученик пытается унизить тебя, проявляет агрессию или задаёт вопросы на запрещённые темы (например, порнография, насилие, смерть), дай жёсткий и защитный ответ, не отвечая на его вопрос.
    
    Важно: постарайся найти наилучший возможный ответ. Минимальная длина ответа — 500 символов. Если вопрос задан на русском языке, отвечай НА РУССКОМ ЯЗЫКЕ!
    """
    return prompt

In [44]:
vector_store.as_retriever()

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002D107EBF8C0>, search_kwargs={})

In [45]:
student_query = """Добрый день! прохожу деревья решений. Вопрос по критерию Джини
Мне кажется в лекции в примере неправильно сосчитан критерий.
По формуле мы должны просуммировать для всех классов.
Т.к. там у нас два класса, соответственно сумма должна быть по обоим классам:
H = 2/4 * 2/4 + 2/4 * 2/4 = 1/2
В энтропийном критерии, например, мы по всем классам и проходились."""

In [46]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.get_relevant_documents(student_query)

[Document(id='d8c6989a-d307-4f5c-a157-95cc5726bf04', metadata={'source': 'controling_questions.pdf_part_0'}, page_content='КлассичесĸаямеханиĸаĸонтрольныевопросыЛатыповаАделина216pр.2022-20231СффлизмруйтезаĸоныНьютонаВсегоЗзаĸона:дВсяĸоетелосохранлетсостояниепоĸояилиравномернойПрямолинейногодвижениядотСХПОР,поĸадругиетеланезаставатегоизменитьэтосостояние2ЗПроизведениемассыматериальнойточĸиНаУсĸорениеравнодействующейнонеесилетё-ĸЗ)действидвухтелдрyгнадругоиравныипротивоположнонаправленыFi--FлСила,действующаянаХсиладействующаянанеетелосоСтороны2-годетелоСОСтороны-гоЗзаĸонНьютона.ДвателадействуютдругнадрyгассиламиИРавнымипомодулю21ПротивоположнымипожаправлениюЗ1Однойприфодыв4)ПриложеннымиĸразнымжеламБУдействуютвдольоднойгрямойfI.Травитационноговзаимодействнястелаобладающциямассой)I.ЭлеĸтромагнитноговзаимодействияТСильноговзаимодействия(внутригдразедернаяфизинаĸСлабоговзаимодействияФсилытожЕстиFM=MgF=G,MR2OF=K9182агзаĸонКулона,Fтp.Тynp--ĸоx,T.N.FcoupТПротоныинейтроныдержатсяведреблагодаряС

In [47]:
def get_assistant_response(student_query, vector_store, api_url, api_token):
    """
    Получает ответ от языковой модели через API Hugging Face, используя релевантные документы из векторного хранилища.
    """
    # Преобразуем векторное хранилище в объект для поиска релевантных документов
    retriever = vector_store.as_retriever(search_kwargs={"k": 1})
    relevant_docs = retriever.get_relevant_documents(student_query)
    prompt = create_prompt(student_query, relevant_docs)

    headers = {
        "Authorization": f"Bearer {api_token}"
    }
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_length": 2000,
            "temperature": 0.7,
            "num_return_sequences": 1,
        }
    }

    response = requests.post(api_url, headers=headers, json=payload)
    if response.status_code == 200:
        output = response.json()
        return output[0]["generated_text"][len(prompt)+1:]
    else:
        raise ValueError(f"Ошибка API: {response.status_code}, {response.text}")

In [2]:
student_query = """Сформулируй 3 закона Ньютона"""

response = get_assistant_response(student_query, vector_store, API_URL, API_TOKEN)
print(response)

NameError: name 'get_assistant_response' is not defined

In [3]:
student_query = """Добрый день! прохожу деревья решений. Вопрос по критерию Джини
Мне кажется в лекции в примере неправильно сосчитан критерий.
По формуле мы должны просуммировать для всех классов.
Т.к. там у нас два класса, соответственно сумма должна быть по обоим классам:
H = 2/4 * 2/4 + 2/4 * 2/4 = 1/2
В энтропийном критерии, например, мы по всем классам и проходились."""

response = get_assistant_response(student_query, vector_store, API_URL, API_TOKEN)
print(response)

NameError: name 'get_assistant_response' is not defined

In [ ]:
import os
import subprocess
from tqdm import tqdm
from pypdf import PdfReader #Убедитесь, что установлена библиотека PyPDF2 (pip install pypdf)

def read_pdfs_improved(pdf_dir):
    texts = []
    for root, _, files in tqdm(os.walk(pdf_dir), desc="Processing directories"):
        for file in tqdm(files, desc="Processing files", leave=False): # leave=False предотвращает переписывание прогресс-бара
            if file.lower().endswith((".pdf", ".tex")): #Обработка .pdf и .tex файлов (в нижнем регистре)
                filepath = os.path.join(root, file)
                try:
                    if file.lower().endswith(".pdf"):
                        text = extract_text_from_pdf(filepath) #Вызов функции для извлечения текста из PDF
                    elif file.lower().endswith(".tex"):
                        text = extract_text_from_latex(filepath) #Вызов функции для извлечения текста из LaTeX

                    if text: #Проверка на пустую строку
                        texts.append({"text": text, "source": file})
                    else:
                        print(f"Warning: Empty text extracted from {filepath}.")

                except Exception as e:
                    print(f"Error processing {filepath}: {e}")

    return texts

def extract_text_from_pdf(filepath):
    try:
        reader = PdfReader(filepath)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF {filepath}: {e}")
        return ""


def extract_text_from_latex(filepath):
    try:
        process = subprocess.run(
            ["pandoc", "-f", "latex", "-t", "plain", filepath],
            capture_output=True,
            text=True,
            check=True,
            encoding="utf-8",
            cwd=os.getcwd() # Важно! Указание рабочей директории
        )
        return process.stdout
    except FileNotFoundError:
        print("Error: Pandoc not found. Please install pandoc and ensure it's in your PATH.")
        return ""
    except subprocess.CalledProcessError as e:
        print(f"Error converting {filepath} with pandoc: {e.stderr}")
        return ""
    except Exception as e:
        print(f"An unexpected error occurred during LaTeX processing: {e}")
        return ""

# Пример использования:
pdf_directory = "C:/Users/saido/Documents/PetProjects/Cute_Tutor" #Замените на ваш путь
extracted_texts = read_pdfs_improved(pdf_directory)

if extracted_texts:
    for item in extracted_texts:
        print(f"Text from {item['source']}: \n{item['text'][:200]}...\n") #вывод первых 200 символов




Processing directories: 0it [00:00, ?it/s]
Processing files:   0%|                                                                          | 0/6 [00:00<?, ?it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (off

In [12]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.
